
## MNIST PyTorch - RNN LSTM

* Simple recurrent neural network for mnist
* To predict the class per 28x28 image, we now think of the image as a sequence of rows. 
* Therefore, you have 28 rows of 28 pixels each


In [1]:

import torch
import numpy as np
import os

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
import pandas as pd

from numpy import genfromtxt

from PIL import Image


In [2]:

import sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

from mlxtend.plotting import heatmap
from sklearn.model_selection import train_test_split



In [3]:

from torch.utils.data import TensorDataset, DataLoader

import torch.optim as optim 
import torch.nn as nn


import torch.nn.functional as F

from torch.autograd import Variable



## Parameters


In [4]:

learning_rate    = 0.003  ## Adam default   ## 0.001
batch_size       = 1000   ## 32
N_Epochs         = 20  ##27000  

seq_len          = 28     # MNIST data input (img shape: 28*28)
size_of_vector   = 28     # chunks per image
rnn_hidden_size  = 128    # size of rnn
n_classes        = 10     # MNIST total classes (0-9 digits) ## B



## Read the data


In [5]:

data_path = "data/MNISTdata/"


mnist_train = datasets.MNIST(data_path, train=True, download=True)


In [6]:

mnist_test = datasets.MNIST(data_path, train=False, download=True)


In [7]:

mnist_train_tr = datasets.MNIST(data_path, train=True, download=False, 
                                            transform=transforms.Compose([
                                                transforms.ToTensor()
                                            ]))
mnist_test_tr  = datasets.MNIST(data_path, train=False, download=False, 
                                            transform=transforms.Compose([
                                                transforms.ToTensor()
                                            ]))



## View the Images


In [8]:

Image.fromarray(  mnist_test_tr.data[423].numpy()  ) 


In [9]:

## mnist_train_tr.data = mnist_train_tr.data.view(60000, 1, 28, 28)

mnist_train_tr.data.shape


torch.Size([60000, 28, 28])


## Create Data Loaders


In [10]:

mnist_train_tr.data.shape


torch.Size([60000, 28, 28])

In [11]:

mnist_test_tr.data.shape


torch.Size([10000, 28, 28])

In [12]:


train_dl  = torch.utils.data.DataLoader(mnist_train_tr, batch_size=batch_size, shuffle=True  ) 
test_dl   = torch.utils.data.DataLoader(mnist_test_tr,  batch_size=batch_size, shuffle=False ) 



## Utility Functions


In [13]:

def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    f1_measure = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')
    print('F1-mesure: %.3f' % f1_measure)
    return f1_measure, confmat 


In [14]:

def plot_metric_per_epoch(the_scores_list):
    x_epochs = []
    y_epochs = [] 
    for i, val in enumerate(the_scores_list):
        x_epochs.append(i)
        y_epochs.append(val)
    
    plt.scatter(x_epochs, y_epochs,s=50,c='lightgreen', marker='s', label='score')
    plt.xlabel('epoch')
    plt.ylabel('score')
    plt.title('Score per epoch')
    plt.legend()
    plt.grid()
    plt.show()



## Convert Batches to sequence

* batch_x has tensors of 28x28 which need to be converted to a sequence
* Reshape data to get 28 in seq_len, N_batches, 28 features in vector



In [15]:
## MNIST data input (img shape: 28*28)
## seq_len         = 28     
## size_of_vector  = 28     
## batch_size     

## Permute converts to [seq_len=28, batch_size, size_of_vector=28]

def make_img_to_sequence(batch_x):
      
    batch_x_new = batch_x.permute(1, 0, 2)
    
    return batch_x_new



## Main Loop


In [16]:

def training_loop( N_Epochs, model, loss_fn, opt  ):
    
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:
            
            xb = torch.squeeze(xb, dim=1)
                        
            y_pred = model( xb )

            loss   = loss_fn(y_pred, yb)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        if epoch % 1 == 0:
            print(epoch, "loss=", loss)



## LSTM Architecture


In [19]:

class MNIST_LSTM_Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        ##             (  vector_size,  hidden_size, n_layers)
        self.lstm              = nn.LSTM( 28,          128,        1)
        
        self.dropout           = nn.Dropout(0.2)
        self.fully_connected   = nn.Linear(128, 10)
        
    def forward(self, x):
        
        batch_x_seq   = make_img_to_sequence( x )
        h0, c0        = self.init_hidden()      ## the initial hidden state
        lstm_o, hn    = self.lstm(x, h0)
        lstm_o        = self.dropout(lstm_o)
        
        out           = self.fully_connected( hn )

        ## lstm_o = lstm_o.view(-1, self.hidden_dim)
        ## output = self.fc(lstm_o)
        
        y_pred             = out.view( -1,        10) 
        
        return y_pred

    def init_hidden(self):
        ## Initial hidden layer in time is all zeros
        ##       (n_layers,  batch_size, rnn_hidden_size)
        h0 = torch.randn(1,        28,             128)
        c0 = torch.randn(1,        28,             128)
        return h0, c0




## Core Functions


In [20]:

model     = MNIST_LSTM_Net()

loss_fn   = nn.CrossEntropyLoss( )  

opt       = torch.optim.Adam( model.parameters(), lr=learning_rate )

training_loop(  N_Epochs, model, loss_fn, opt  )
    



IndexError: index 1 is out of bounds for dimension 0 with size 1

In [ ]:


f1_scores_to_plot = []

with torch.no_grad():
    for xb, yb in test_dl:
        
        xb = torch.squeeze(xb, dim=1)
        
        y_pred = model(  xb  )
        
        vals, indeces = torch.max( y_pred, dim=1  )
        preds = indeces
        f1, last_conf_mtrx = print_metrics_function(yb, preds)
        f1_scores_to_plot.append(f1)


In [ ]:

plot_metric_per_epoch(f1_scores_to_plot)


In [ ]:

print(last_conf_mtrx )


In [ ]:

sum(f1_scores_to_plot) / len(f1_scores_to_plot)



## LSTM simple example


In [38]:
##                (vector_size,  hidden_size,  n_layers)
rnn_lstm_rc = nn.LSTM(      28,          128,        1)



In [39]:
## the data needs to be permuted
## 
##                     (seq_len,    batch_size,  vector_size)
x_lstm_rc = torch.randn(      28,         1000,           28)

x_lstm_rc.shape


torch.Size([28, 1000, 28])

In [40]:

##            (n_layers,   batch_size,      hidden_size)
h0 = torch.randn(     1,            1000,           128)

c0 = torch.randn(     1,            1000,           128)


In [41]:

output, (hn, cn) = rnn_lstm_rc(x_lstm_rc, (h0, c0))


In [42]:

output.shape


torch.Size([28, 1000, 128])

In [43]:

hn.shape


torch.Size([1, 1000, 128])

In [44]:

cn.shape


torch.Size([1, 1000, 128])


## LSTM sequential


In [45]:


rc_n_hidden   = 128
rc_input_size = 28     
rc_n_layers   = 1
rc_output     = 10


In [46]:

model_lstm_seq = torch.nn.Sequential(
    
    ##           (        28,      128,           1) 
    torch.nn.LSTM(rc_input_size, rc_n_hidden, rc_n_layers),
    
    torch.nn.Dropout(0.2),
    
    ##             (     128,        10)
    torch.nn.Linear(rc_n_hidden, rc_output)
    
)
